#                         Miniscope Data Analysing Software

This software was created to personal lab purposes. It written based on PIMPN-master by Guillaume Etter and based on CaImAn-master python codes and their jupyter notebook version. For detailed use see the CaImAn-documentation and wiki.

# Initialization

Firs of all you need to import any dependencies into memory to start proper analysing.
!!! CAREFUL:
    Before starting jupyter notebook with MiDAS you need to install caiman and to write in the anaconda command line the following command:
    >> conda activate caiman

In [ ]:
from datetime import datetime
import pandas as pd
import scipy.io as sio
import re
import os
import h5py
import csv
import tensorflow as tf
import time
import logging
import holoviews as hv
import zipfile
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('default')
import numpy as np
from moviepy.editor import *
import smtplib
import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import peakutils
from tkinter import filedialog
from tkinter import *
import bokeh.plotting as bpl
try:
       from bokeh.io import vform, hplot
except:
       # newer version of bokeh does not use vform & hplot, instead uses column & row
       from bokeh.layouts import column as vform
       from bokeh.layouts import row as hplot
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.io import export_png

bpl.output_notebook()
hv.notebook_extension('bokeh')

In [ ]:
root = Tk()
Button(root, text="Quit", command=root.destroy).pack()
root.folder =  filedialog.askdirectory(title = "Select folder to analyse")
path = root.folder
root.mainloop()
print('The analyses path is: ' + path)

ido = datetime.today()
ev = str(ido.year)
honap = str(ido.month)
nap = str(ido.day)
experimentName = ev + '_' + honap + '_' + nap
print('The name of experiment is: ' + experimentName)


path_to_analyze = path
analyze_behavior = True
spatial_downsampling = 3 # Drastically speeds up processing. 2-3 recommended
isnonrigid = False

path_to_results = path + '/' + experimentName
try:
    os.mkdir(path_to_results) # Where to save the data
    print("Result directory " , path_to_results , " Created")
except FileExistsError:
    print("Directory: " , path_to_results , " already existed" )

    

print('Parameters saved. Ready to start analyzing')

In [ ]:
path = path_to_analyze
files = [i for i in os.listdir(path) if os.path.isfile(os.path.join(path,i)) and \
     'msCam' in i]
msFileList = [] #create an empty list
for file in files:
    if file.endswith(".avi"):
        msFileList.append(os.path.join(path, file)) #append result to list


    

print('In this folder the number of msCam videos is: ')
print(len(files))
print('-------------------------------------------------------------------------------------------------------------------------------')
fnames = msFileList
print(sorted(fnames))

# Logging

In [ ]:
logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.WARNING)

# Enable paralel processing for faster analysis

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
now = datetime.now()
analysis_time = now.strftime("%Y-%m-%d %H:%M") # This is to register when the analysis was performed
print('Analysis started on ' + analysis_time)


analysis_start = time.time() # This is to register the time spent analyzing

# Motion correction params setup

In [ ]:
# dataset dependent parameters
frate = 20                       # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds
dirExperimentName = path_to_analyze


# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = False         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)     # size of high pass spatial filtering, used in 1p data
max_shifts = (5, 5)      # maximum allowed rigid shift
strides = (48, 48)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)      # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries
use_cuda = True
memory_fact = 1

mc_dict = {
    'fnames': fnames,
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}

opts = params.CNMFParams(params_dict=mc_dict)

# Perform motion correction

In [ ]:
start = time.time()
if motion_correct:
    # do motion correction rigid
    mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    fname_mc = mc.fname_tot_els if pw_rigid else mc.fname_tot_rig
    
end = time.time()
print(end-start)
print('Motion correction has been done!')

# Memory mapping the motion corrected videos

In [ ]:
if motion_correct:
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                     np.max(np.abs(mc.y_shifts_els)))).astype(np.int)
    else:
        bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)

    bord_px = 0 if border_nan is 'copy' else bord_px
    fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
    
else:  # if no motion correction just memory map the file
    fname_new = cm.save_memmap(fnames, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)
    
print('Motion corrected video has been mapped to memory!')

In [ ]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

In [ ]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

# Perform a projection of correlated pixels (and associated signal-to-noise ratio) in motion corrected video
This is important to assess the amounts of local correlations and peak-to-noise ratio as well as seed/initialize CNMFe

In [ ]:
# compute some summary images (correlation and peak to noise)
cn_filter, pnr = cm.summary_images.correlation_pnr(images[::5], gSig=3, swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile


In [ ]:
#Plot the results of the correlation/PNR projection
plt.figure(figsize=(20,10))
plt.subplot(2, 2, 1); plt.imshow(cn_filter); plt.colorbar(); plt.title('Correlation projection')
plt.subplot(2, 2, 2); plt.imshow(pnr); plt.colorbar(); plt.title('PNR')

# Setup CNMFe params 

In [ ]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .65      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 1             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .2       # min peak value from correlation image
min_pnr = 6       # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor
memory_fact = 1


opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px})                # number of pixels to not consider in the borders

# Perform CNMFe

In [ ]:
start = time.time()
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(images)
end = time.time()
print('CNMFe done in: ')
print(end-start)

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 3            # adaptive way to set threshold on the transient size
r_values_min = 0.85    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': True})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

# Plot neuron contours

In [ ]:
#%% plot contour plots of accepted and rejected components
cnm.estimates.plot_contours(img=cn_filter, idx=cnm.estimates.idx_components)

# Plot the result

In [ ]:
%matplotlib inline

#How many neurons to plot
neuronsToPlot = len(cnm.estimates.C)

DeconvTraces = cnm.estimates.S
RawTraces = cnm.estimates.C
SFP = cnm.estimates.A
SFP_dims = list(dims)
SFP_dims.append(SFP.shape[1]) 
print('Spatial foootprints dimensions (height x width x neurons): ' + str(SFP_dims))

numNeurons = SFP_dims[2]

SFP = np.reshape(SFP.toarray(), SFP_dims, order='F')

maxRawTraces = np.amax(RawTraces)

plt.figure(figsize=(30,15))
plt.subplot(341);
plt.subplot(345); plt.plot(mc.shifts_rig); plt.title('Motion corrected shifts')
plt.subplot(3,4,9);
plt.subplot(3,4,2); plt.imshow(cn_filter); plt.colorbar(); plt.title('Correlation projection')
plt.subplot(3,4,6); plt.imshow(pnr); plt.colorbar(); plt.title('PNR')
plt.subplot(3,4,10); plt.imshow(np.amax(SFP,axis=2)); plt.colorbar(); plt.title('Spatial footprints')

plt.subplot(2,2,2); plt.figure; plt.title('Raw traces')
plot_gain = 10 # To change the value gain of traces
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(RawTraces[i,:],'k')
    else:
      trace = RawTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(RawTraces[i,:],'k')
    else:
      trace = RawTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')

plt.subplot(2,2,4); plt.figure; plt.title('Deconvolved traces')
plot_gain = 20 # To change the value gain of traces
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(DeconvTraces[i,:],'k')
    else:
      trace = DeconvTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(DeconvTraces[i,:],'k')
    else:
      trace = DeconvTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')      

# Save summary figure
plt.savefig(path_to_results + '/' + 'summary_figure.svg', edgecolor='w', format='svg', transparent=True)

#  Fancy way to show the results

In [ ]:
# accepted components and it's traces
CI = cm.local_correlations(images[::1].transpose(1,2,0))
CI[np.isnan(CI)] = 0
cnm.estimates.nb_view_components(img=CI, idx=cnm.estimates.idx_components, cmap = 'gray')
# if you want to see the rejected components too, delete idx=....

In [ ]:
# this shows actually the deconvolved version of the traces
cnm.estimates.detrend_df_f(quantileMin=8, frames_window=250)
cnm.estimates.select_components(use_object=True)
cnm.estimates.nb_view_components(img=cn_filter, denoised_color='red', cmap = 'gray')

In [ ]:
cnm.estimates.select_components(use_object=True)

# Delete unwanted neurons

In [ ]:
n = len(cnm.estimates.C)
j = [0] * n
for k in range(len(cnm.estimates.C)):
    plt.figure()
    plt.plot(RawTraces[k,:])
    plt.pause(0.2)
    j[k] = input("Is it a good trace or not? If yes press 1, if not press 0!:")

In [ ]:
print(j)
res = [idx for idx, val in enumerate(j) if val != '0']

In [ ]:
n = len(res)
FinalTraces = [0] * n
FinalDeconvs = [0] * n
for m in range(n):
    h = res[m]
    FinalTraces[m] = RawTraces[h]
    FinalDeconvs[m] = DeconvTraces[h]

# Plot again the datas without the bad neurons 

In [ ]:

neuronsToPlot = len(FinalTraces)
plt.subplot(2,2,1); plt.figure; plt.title('Final Raw traces')
plot_gain = 10
maxFinalTraces = np.amax(FinalTraces)
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(FinalTraces[i])
    else:
      trace = FinalTraces[i] + maxFinalTraces*i/plot_gain
      plt.plot(trace)
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(FinalTraces[i])
    else:
      trace = FinalTraces[i] + maxFinalTraces*i/plot_gain
      plt.plot(trace)

plt.subplot(2,2,2); plt.figure; plt.title('Final Deconvolved traces')
plot_gain = 10 # To change the value gain of traces
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(FinalDeconvs[i])
    else:
      trace = FinalDeconvs[i] + maxFinalTraces*i/plot_gain
      plt.plot(trace)
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(FinalDeconvs[i])
    else:
      trace = FinalDeconvs[i] + maxFinalTraces*i/plot_gain
      plt.plot(trace,'k')
        
        
# Save summary figure
plt.savefig(path_to_results + '/' + 'finaltraces.svg', edgecolor='w', format='svg', transparent=True)        

# Stop the process

In [ ]:
# Stop counter and register analysis time
analysis_end = time.time()

analysis_duration = analysis_end - analysis_start

print('Done analyzing. This took a total ' + str(analysis_duration) + ' s')

# Save results in hdf5 file

In [ ]:
save_hdf5 = True
if save_hdf5: 
    cnm.save(path_to_results + '/' + 'analysis_results.hdf5')

# Save results in Matlab format 

In [ ]:
from scipy.io import savemat
save_mat = True
if save_mat:  
    results_dict = {
                'dirName': path_to_analyze,
                'numFiles': len(msFileList),
                'framesNum': len(RawTraces[1]),
                'maxFramesPerFile': 1000,
                'height': dims[0],
                'width': dims[1],
                'Experiment': experimentName,
                'camNumber': 0,
                'analysis_time': analysis_time,
                'ds': spatial_downsampling,
                'shifts': mc.shifts_rig,
                'meanFrame': [], #TO DO
                'Centroids': [], #TO DO
                'CorrProj': cn_filter,
                'PeakToNoiseProj': pnr,
                'FiltTraces': FinalTraces,
                'Decoaccepted': FinalDeconvs,
                'RawTraces': RawTraces.conj().transpose(), #swap time x neurons dimensions
                'SFP': SFP,
                'numNeurons': SFP_dims[2],
                'DeconvTraces': DeconvTraces
                }

    SFPperm = np.transpose(SFP,[2,0,1])
    sio.savemat(path_to_results + '/SFP.mat', {'SFP': SFPperm})
    sio.savemat(path_to_results + '/ms.mat', {'ms': results_dict})

# Behavior

In [ ]:
from caiman.behavior import behavior
import caiman as cm

In [ ]:
import os
from tkinter import filedialog
from tkinter import *
import re
root2 = Tk()
Button(root2, text="Quit", command=root2.destroy).pack()
root2.folder =  filedialog.askdirectory(title = "Select folder to analyse")
path = root2.folder
root2.mainloop()

In [ ]:
bfiles = [i for i in os.listdir(path) if os.path.isfile(os.path.join(path,i)) and \
     'behavCam' in i]
bFileList = [] #create an empty list
print(bfiles)
k = len(bfiles)


In [ ]:
for bfile in bfiles:
    if bfile.endswith(".avi"):
        bFileList.append(os.path.join(path, bfile)) #append result to list

print('In this folder the number of msCam videos is: ')
print(len(bfiles))


bfnames = bFileList
print(bfnames)

In [ ]:
print('All the data successfully analized')